In [1]:
import numpy as np
import pandas as p

#reduce original train and events to 1/5 its' size to not exceed 1gb limit on github
#split clicks_train into train and test for good, since we havent the labels of clicks_test (original test)
#split by timestamp, first 3/4 train, following 1/4 test, same as was in original train/test

clicks_train = p.read_csv("/Users/astrachan/Desktop/history/clicks_train.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})

In [2]:
print clicks_train.head()

   display_id   ad_id clicked
0           1   42337   False
1           1  139684   False
2           1  144739    True
3           1  156824   False
4           1  279295   False


In [3]:
display_ids = clicks_train.groupby(['display_id'])['display_id'].agg({'count' : 'count'}).reset_index().drop('count',axis = 1)

In [4]:
print display_ids.head()

   display_id
0           1
1           2
2           3
3           4
4           5


In [5]:
reduction = display_ids.shape[0] / 5

In [6]:
split = reduction * 3 / 4

In [7]:
train = clicks_train.merge(display_ids[:split], how = 'inner', on = 'display_id')
test = clicks_train.merge(display_ids[split:reduction], how = 'inner', on = 'display_id')

In [8]:
print train.tail()
print test.head()
print train.shape
print test.shape

          display_id   ad_id clicked
13104204     2531187  192990   False
13104205     2531187  252434   False
13104206     2531188  213064   False
13104207     2531188  220463   False
13104208     2531188  371779    True
   display_id   ad_id clicked
0     2531189   89423    True
1     2531189   93224   False
2     2531189  176615   False
3     2531189  187283   False
4     2531190   73651   False
(13104209, 3)
(4329331, 3)


In [9]:
train.to_csv('/Users/astrachan/Desktop/ludast_outbrain/ludast_outbrain/source_tables/built/train.csv', index=False)
test.to_csv('/Users/astrachan/Desktop/ludast_outbrain/ludast_outbrain/source_tables/built/test.csv', index=False)

In [10]:
#reduce events as well, not getting uuids since we don't use them and will also free some space
events = p.read_csv("/Users/astrachan/Desktop/history/events.csv", usecols = ["display_id", "document_id", "timestamp", "platform", "geo_location"], dtype={"display_id":int, "document_id":int, "timestamp":int, "platform":str, "geo_location":str})

In [11]:
events.head()

,display_id,document_id,timestamp,platform,geo_location
0,1,379743,61,3,US>SC>519
1,2,1794259,81,2,US>CA>807
2,3,1179111,182,2,US>MI>505
3,4,1777797,234,2,US>WV>564
4,5,252458,338,2,SG>00


In [12]:
events = events.merge(display_ids[:reduction], how = 'inner', on = 'display_id')

In [13]:
events.to_csv('/Users/astrachan/Desktop/ludast_outbrain/ludast_outbrain/source_tables/built/events.csv', index=False)